## Import Packages

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

##for plotting
import matplotlib.pyplot as plt
import seaborn as sns

from statsmodels.graphics.tsaplots import plot_acf


#for csv's
import os
import glob
import re

## Functions Used

In [2]:
## Convert the data frame index to a datetime index
def to_datatime_index(data, file_name):

    d_time = data.time.values

    ## Hold 3 digits
    d_time_string = [str(time)[0:10] for time in d_time]

    timestamp_time = [datetime.strptime(x,"%H%M%S.%f") for x in d_time_string]

    ## Add date
    date_time = [ timestamp_time[i].replace(year = int(file_name[8:12]), month = int(file_name[12:14]), day = int(file_name[14:16]))
     for i in range(data.shape[0]) ]

    ## Set timestamp index to the entire data
    data.index = date_time

    ## Replace old 'time' column
    data.drop(['time'], axis = 1, inplace = True)

    data.index.name = 'Date-time'

    return data

In [3]:
def choose_months(t_0,t_1):
    
    out = []
    
    ## Make list of the numbers of months
    if (t_0[0] == '0') & (t_1[0] == '0'):
        
        month_nums = ['0{}'.format(str(x)) for x in range(int(t_0),int(t_1)+1)]
    
    elif (t_0[0] == '0'):
        
        s_1 = ['0{}'.format(str(x)) for x in range(int(t_0),10)]

        s_2 = ['{}'.format(str(x)) for x in range(10,int(t_1)+1)]

        month_nums =  s_1 + s_2
    
    else:
        
        month_nums =  ['{}'.format(str(x)) for x in range(int(t_0),int(t_1)+1)]
        
    ## Fill in the empty 'out' with
    ## all selected months.
    for num in month_nums:
        
        out.append(select_month(num))
        
     ## Create the final dataframe by
    ## concatination of out
    return pd.concat(out)

In [4]:
def select_month(month):
    
    select = []
    
    for i in range(len(all_filenames)):
    
        if all_filenames[i][12:14] == str(month):

            temp = pd.read_csv(all_filenames[i])
    
            temp = to_datatime_index(data = temp, file_name = str(all_filenames[i]))
        
            select.append(temp)
            
    return pd.concat(select)

In [5]:
def get_mean_t(prices, weights):
    
    temp = weights.mul(prices.values)
    
    out = pd.DataFrame(temp.sum(axis = 1))
    
    out.columns = ['W.Mean']
    
    return out 

In [6]:
def get_var_t(in_prices, in_weights):
    
    w_mean = get_mean_t(in_prices, in_weights)
    
    temp = in_prices.sub(w_mean.values)

    temp2 = temp.pow(2)

    temp3 = in_weights.mul(temp2.values)
    
    out = pd.DataFrame(temp3.sum(axis = 1))
    
    out.columns = ['W.Var']
    
    return out

In [7]:
def get_data():
    
    time = input('Day, Month or Period?')
    
    if time == 'd':

        date = input('Give date in month/date XXXX format')
        
        name =  'GARAN.E_2017{}.csv'.format(str(date))
    
        data = pd.read_csv('GARAN.E_2017{}.csv'.format(str(date)))
        
        data = to_datatime_index(data, name)

    elif time == 'm':
    
        month_input = str(input('Give month : '))
    
        data = select_month(month_input)  
    
    elif time == 'p':
    
        start = input('Start month : ')
        end = input('End month : ')
    
        data = choose_months(start,end)  
        
    return data

In [8]:
def disp_stats(data):
    
    bids_t = data[['bid1','bid2','bid3','bid4','bid5']]

    bsizes_t = data[['bsize1','bsize2','bsize3','bsize4','bsize5']]

    asks_t = data[['ask1','ask2','ask3','ask4','ask5']]

    asizes_t = data[['asize1','asize2','asize3','asize4','asize5']]
    
    bids_max_t = bids_t.max(axis = 1)
    
#     bids_max_vol_t = bsizes_t[]
    
    asks_min_t = asks_t.min(axis = 1)
    
    ## Calculate bid/ask weights:
    ## Divide each row (axis = 0) by the 
    ## corresponding row sum (axis = 1)of bsizes 
    bweights = bsizes_t.div(bsizes_t.sum(axis = 1), axis = 0)

    aweights = asizes_t.div(asizes_t.sum(axis = 1), axis = 0)
    
    midpoint_price = (bids_max_t+asks_min_t)/2
    
    bid_w_mean = get_mean_t(bids_t, bweights)
    
    bids_w_var = get_var_t(bids_t, bweights)
    
    asks_w_mean = get_mean_t(asks_t, aweights)
    
    asks_w_var = get_var_t(asks_t, aweights)
    
    ## Spread = difference between best bid and ask price for every t
    spread = pd.DataFrame(asks_t.min(axis = 1) - bids_t.max(axis = 1))
    
    out =  pd.concat([midpoint_price, 
                     bid_w_mean, bids_w_var, 
                     asks_w_mean, asks_w_var, 
                     bids_max_t, asks_min_t, 
                     spread], axis = 1)
    
    out.columns = ['Midpoint Price','Bids W.Mean', 'Bids W.Var', 'Asks W.Mean', 'Asks W.Var', 'Best Bid', 'Best Ask','Spread']
    
#     out['Spread'] = diff.values
    
    return out

In [9]:
def from_mult_index_to_dt(data):

    timestamp_time = [datetime(year = 2017,
                     month = data.index[row][0], 
                     day = data.index[row][1],
                     hour = data.index[row][2],
                     minute = data.index[row][3]) for row in range(len(data.index))]
    
    data.index = timestamp_time
    data.index.name = 'Date-time'
    
    return data

In [10]:
def change_freq(df, freq = '1min'):
    
    ls_out = []
    ind_out = []
    
    ## Create Multiindex for Y,M,D,h,m,s,ms
    grouped = user_data.groupby([user_data.index.year, 
                             user_data.index.month, 
                             user_data.index.day, 
                             user_data.index.hour, 
                             user_data.index.minute,
                             user_data.index.second,
                             user_data.index.microsecond]).first()

    grouped.index.names = ['Year',
                           'Month',
                           'Day',
                           'Hour',
                           'Minute',
                           'Second',
                           'Microsecond'] 
    
    for month in np.unique(grouped.index.get_level_values('Month')):
        data_month = grouped[grouped.index.get_level_values('Month') == int(month)]
        data_days_in_month = data_month.loc[2017,int(month),]
        
        for day in np.unique(data_days_in_month.index.get_level_values('Day')):
            data_day = data_days_in_month[data_days_in_month.index.get_level_values('Day') == day]
            data_hour = data_day.loc[day,]
    
            for hour in np.unique(data_hour.index.get_level_values('Hour')):
                data_minute = grouped[grouped.index.get_level_values('Minute') == int(hour)]
                data_minute = data_hour.loc[int(hour),]
    
                for minute in np.unique(data_minute.index.get_level_values('Minute')):
                    data_sec = data_minute.loc[minute,]
                    ## Store the last value of the last second of minute
                    ls_out.append(data_sec.tail(1))
                    ind_out.append((month,day,hour,minute))

    ls_out = pd.concat(ls_out)
    ls_out.index = pd.MultiIndex.from_tuples(ind_out, names = ['Month','Day','Hour','Minute'])

    return from_mult_index_to_dt(ls_out)

In [11]:
def get_densities(data, what):
    
    if str(what) == 'bid':
        bids = data[['bid1','bid2','bid3','bid4','bid5']]
        bsize = data[['bsize1','bsize2','bsize3','bsize4','bsize5']]
        total_bsize = bsize.sum(axis = 1)
        bweights = bsize.div(total_bsize, axis = 0)
        out =  pd.concat([bids, bweights, total_bsize], axis = 1)
        out.columns = ['Bid 1','Bid 2','Bid 3','Bid 4','Bid 5',
                       'Bid Weight 1','Bid Weight 2','Bid Weight 3','Bid Weight 4','Bid Weight 5',
                       'Total Size']
    
    elif str(what) == 'ask':
        asks = data[['ask1','ask2','ask3','ask4','ask5']]
        asize = data[['asize1','asize2','asize3','asize4','asize5']]
        total_asize = asize.sum(axis = 1)
        aweights = asize.div(total_asize, axis = 0)
        out =  pd.concat([asks, aweights, total_asize], axis = 1)
        out.columns = ['Ask 1','Ask 2','Ask 3','Ask 4','Ask 5',
                       'Ask Weight 1','Ask Weight 2','Ask Weight 3','Ask Weight 4','Ask Weight 5',
                       'Total Size']
        
    return out

In [12]:
def moments(data, up_to):
    
    store = []
    
    for i in range(1, up_to + 1):
        store.append(data.iloc[:,:5].pow(i,axis = 1).mul(data.iloc[:,5:10].values, axis = 1).sum(axis = 1))
        
    out = pd.concat(store, axis = 1)
    
    out.columns = ['1st Moment','2nd Moment','3rd Moment','4th Moment']
    
    return out

In [13]:
# Data must be in minutes
def change_freq2(data):
    
    days = np.unique(data.index.strftime("%Y-%m-%d"))
    
    out = []
    
    for day in days:
        
        ind1 = pd.date_range(start = '{} 10:10:00'.format(day),end = '{} 12:50:00'.format(day), freq = 'min')
        ind2 = pd.date_range(start = '{} 14:10:00'.format(day),end = '{} 17:50:00'.format(day), freq = 'min')

        correct_ind = ind1.append(ind2)
        
        df1 = data.loc[day][(data.loc[day].index.strftime("%H:%M:%S") >= '10:10') 
                                 & (data.loc[day].index.strftime("%H:%M:%S") <= '12:51')]

        df2 = data.loc[day][(data.loc[day].index.strftime("%H:%M:%S") >= '14:10') 
                                & (data.loc[day].index.strftime("%H:%M:%S") <= '17:51')]

        df3 = pd.concat([df1,df2])
        
        temp_df = pd.DataFrame(index = correct_ind, columns=df3.columns)

        ind = 0
        j = 0
        
        while ind < len(correct_ind):

            while j < len(df3.index):

                if temp_df.index.strftime("%H:%M:%S")[ind] == df3.index.strftime("%H:%M:%S")[j]:
                    temp_df.iloc[ind] = df3.iloc[j]
                    ind += 1
                    j += 1
                else:
                    temp_df.iloc[ind] = df3.iloc[j-1]
                    ind += 1
                    
        out.append(temp_df)
        
            
    return pd.concat(out)  

## Read Data

Sort the csv files in the data folder

In [14]:
file_extension = ".csv"

all_filenames = [i for i in glob.glob(f"*{file_extension}")]

all_filenames.sort(key=lambda c : list(
    map(int, re.findall(r'\d+', c)))[0])

Type in "d" : for specific date in 2017-xx-xx form

Type in "m" : for specific month in xx-form

Type in "p" : for period from start month to end month in xx-form

In [16]:
all_filenames

[]

In [15]:
user_data = get_data()

Day, Month or Period?p
Start month : 01
End month : 12


ValueError: No objects to concatenate

Drop the "order" columns for bids and asks

In [ ]:
user_data = user_data.drop(['bord1','bord2','bord3','bord4','bord5',
               'aord1','aord2','aord3','aord4','aord5'], axis = 1)

In [ ]:
user_data.head()

In [ ]:
out = {}

for day in np.unique(user_data.index.strftime("%Y-%m-%d")):
    print('Date : {} \n Number of data points : {}'.format(str(day), user_data.loc[day].shape[0]))
    out[str(day)] = user_data.loc[day].shape[0]

In [ ]:
num_samples = pd.DataFrame(out.values(), index = out.keys())

num_samples.describe().transpose()

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=1, sharex=False, sharey=False, figsize=(12,8))

axes[0].scatter([i for i in range(0,len(out))], out.values())
axes[0].set_xlabel('Days')
axes[0].set_ylabel('Sample size')
axes[1].hist( out.values())
axes[1].set_xlabel('Sample size')
axes[1].set_ylabel('Frequency (%)')
#plt.savefig('sample_frequency2.png')

plt.show()

## Lower Frequency in minutes

In [ ]:
data_year_in_min = change_freq(user_data)

In [ ]:
## Exclude half day 2017-08-31
data_year_in_min = data_year_in_min[data_year_in_min.index.strftime("%Y-%m-%d") != '2017-08-31']

In [ ]:
## Cut into the time intervals and obtain the final data form
data_in_min = change_freq2(data_year_in_min)

In [ ]:
data_in_min[data_in_min.index.hour == 10].head()

## Analysis

Densities of bid and ask

In [ ]:
bids_dens = get_densities(data_in_min, 'bid')
bids_dens

asks_dens = get_densities(data_year_in_min, 'ask')
asks_dens

Calculate the first four moments

In [ ]:
bids_moments = moments(bids_dens, 4)
bids_moments

asks_moments = moments(asks_dens, 4)
asks_moments

Correlation grid for bid and ask moments

In [ ]:
fig, axs = plt.subplots(4, 4, figsize = (16,16))

corrMatrix = np.empty(shape = (4,4), dtype = float)

for i in range(4):
    for j in range(4):
        
        axs[i, j].scatter(bids_moments.iloc[:,i], asks_moments.iloc[:,j], s = 1)             
        axs[i, j].set_title('Bid Moment {} vs Ask Moment {}'.format(i+1,j+1))
        
        ##Build Correlation Matrix
        corrMatrix[i,j] = bids_moments.iloc[:,i].corr(asks_moments.iloc[:,j])


Correlation matrix for the first 4 bid and ask moments

In [ ]:
corr_df = pd.DataFrame(corrMatrix, index = ['Bid Moment {}'.format(i) for i in range(1,5)],
                                   columns = ['Ask Moment {}'.format(i) for i in range(1,5)])

## Midpoint Price

Get data statistics

In [ ]:
stats_in_min = disp_stats(data_in_min)

Plot monthly midpoint price

In [ ]:
import calendar
# jtplot.style()

fig, ax = plt.subplots(figsize = (20,12)) # Create a figure containing a single axes.


for month in np.unique(stats_in_min.index.month):
    
    ax.plot(#stats_in_min[stats_in_min.index.month == month].index,
            stats_in_min[stats_in_min.index.month == month]['Midpoint Price'].values, 
            label = calendar.month_name[month])  # Plot some data on the axes
    ax.set_xlabel('Days')
    ax.set_ylabel('Price')
            

ax.legend()
# major_ticks = np.arange(0,31,4)
# ax.set_xticks(major_ticks)
plt.title('Yearly Midpoint Price for each month per minute')
plt.savefig('midprice_Year_Minutes.png')
plt.show()

Descriptive statistics for every month

In [ ]:
means = [] 
stds = []
Max = []
Min = []

for month in np.unique(stats_in_min.index.month): 
    means.append(stats_in_min[stats_in_min.index.month == month]['Midpoint Price'].describe()['mean'])
    stds.append(stats_in_min[stats_in_min.index.month == month]['Midpoint Price'].describe()['std'])
    Max.append(stats_in_min[stats_in_min.index.month == month]['Midpoint Price'].describe()['max'])
    Min.append(stats_in_min[stats_in_min.index.month == month]['Midpoint Price'].describe()['min'])
        
descriptives_df = pd.DataFrame([means,stds,Max,Min]).transpose()
descriptives_df.index = [calendar.month_name[i] for i in range(1,13)]
descriptives_df.columns = ['Mean','Std','Max','Min']
descriptives_df

Plot midpoint price for the whole year

In [ ]:
fig, ax = plt.subplots(figsize = (20,12)) # Create a figure containing a single axes.
    
ax.plot(stats_in_min['Midpoint Price'].index,
        stats_in_min['Midpoint Price'].values)
ax.set_xlabel('Time')
ax.set_ylabel('Midpoint Price')
            
plt.show()